In [1]:
%load_ext lab_black
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd

In [3]:
src_store_sales = "../data/csv/store_sales.csv"
src_store = "../data/csv/store.csv"
src_items = "../data/csv/item.csv"
src_date = "../data/csv/date_dim.csv"
src_time = "../data/csv/time_dim.csv"

In [12]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").appName("EDA").getOrCreate()
spark.conf.set("spark.sql.debug.maxToStringFields", 1000)
spark.sparkContext.setLogLevel("WARN")

In [13]:
def init_spark(app_name):
    spark = SparkSession.builder.master("local[*]").appName(app_name).getOrCreate()
    return spark


def denormalize_store_sales(df_store, df_store_sales, df_items, df_date, df_time):
    df = (
        df_store_sales.join(
            df_store,
            df_store_sales["ss_store_sk"] == df_store["s_store_sk"],
            "leftouter",
        )
        .join(
            df_items, df_store_sales["ss_item_sk"] == df_items["i_item_sk"], "leftouter"
        )
        .join(
            df_date,
            df_store_sales["ss_sold_date_sk"] == df_date["d_date_sk"],
            "leftouter",
        )
        .join(
            df_time,
            df_store_sales["ss_sold_time_sk"] == df_time["t_time_sk"],
            "leftouter",
        )
    )
    return df


def save_to_csv(df, dest):
    df.write.csv(dest)
    # df.repartition(1).write.csv(dest, sep="|")

In [6]:
df_store_sales = spark.read.option("header", "true").csv(src_store_sales)
df_store = spark.read.option("header", "true").csv(src_store)
df_items = spark.read.option("header", "true").csv(src_items)
df_date = spark.read.option("header", "true").csv(src_date)
df_time = spark.read.option("header", "true").csv(src_time)

In [7]:
df = denormalize_store_sales(df_store, df_store_sales, df_items, df_date, df_time)

In [9]:
df.count()

14400052

In [8]:
assert len(df.schema.names) == (
    len(df_store_sales.schema.names)
    + len(df_store.schema.names)
    + len(df_items.schema.names)
    + len(df_date.schema.names)
    + len(df_time.schema.names)
)

In [14]:
dest = "../data/csv/store_sales_denormalized.csv"
save_to_csv(df, dest)